## 1. Importar Librerías

Importamos:
- `HostedCodeInterpreterTool`: Herramienta para ejecutar código
- `AzureAIAgentClient`: Cliente para crear agentes en Azure AI
- `AzureCliCredential`: Autenticación con Azure CLI

In [ ]:
import asyncio
import os
from agent_framework import HostedCodeInterpreterTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

## 2. Cargar Configuración

Cargamos variables de entorno:
- `AZURE_PROJECT_ENDPOINT`: URL del proyecto de Azure AI Foundry
- `MODEL`: Nombre del deployment del modelo

In [ ]:
load_dotenv()

print(f"✅ Configuración cargada")
print(f"Endpoint: {os.getenv('AZURE_PROJECT_ENDPOINT')}")
print(f"Modelo: {os.getenv('MODEL')}")

## 3. Crear el Agente con Code Interpreter

### HostedCodeInterpreterTool:
- Entorno de ejecución aislado
- Sandbox seguro para código
- Soporta bibliotecas comunes de Python
- Timeout y límites de recursos

### Agente:
- **name**: "CodingAgent"
- **instructions**: Describe sus capacidades
- **tools**: `HostedCodeInterpreterTool()` para ejecutar código

### Contextos async:
Usamos múltiples `async with` para gestionar recursos:
1. `AzureCliCredential()`: Credenciales
2. `AzureAIAgentClient.create_agent()`: Cliente y agente

In [ ]:
async def create_coding_agent():
    """Crea un agente con capacidad de ejecutar código Python"""
    
    print("\n" + "="*70)
    print("CREANDO CODING AGENT")
    print("="*70)
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential, 
            project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
            model_deployment_name=os.getenv("MODEL")
        ).create_agent(
            name="CodingAgent",
            instructions="You are a helpful assistant that can write and execute Python code.",
            tools=HostedCodeInterpreterTool()
        ) as agent,
    ):
        print("\n✅ CodingAgent creado")
        print("   - Nombre: CodingAgent")
        print("   - Herramienta: HostedCodeInterpreterTool")
        print("   - Capacidad: Escribir y ejecutar código Python")
        
        return agent

print("✅ Función create_coding_agent() definida")

## 4. Ejecutar una Tarea de Código

### Tarea:
"Calculate the factorial of 20 using Python code."

### Procesamiento esperado:
1. LLM entiende que necesita código
2. Genera código Python para factorial
3. Llama a `HostedCodeInterpreterTool`
4. Código se ejecuta en sandbox
5. Resultado se retorna
6. LLM incorpora resultado en respuesta

### Ejemplo de código generado:
```python
import math
result = math.factorial(20)
print(result)
```

In [ ]:
async def main():
    """Ejecuta el agente con una tarea de código"""
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential, 
            project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
            model_deployment_name=os.getenv("MODEL")
        ).create_agent(
            name="CodingAgent",
            instructions="You are a helpful assistant that can write and execute Python code.",
            tools=HostedCodeInterpreterTool()
        ) as agent,
    ):
        task = "Calculate the factorial of 20 using Python code."
        
        print("\n" + "="*70)
        print("EJECUTANDO CODING AGENT")
        print("="*70)
        print(f"\n❓ Tarea: {task}")
        print("\n🔄 Procesando...")
        print("   1. LLM genera código Python")
        print("   2. HostedCodeInterpreterTool ejecuta el código")
        print("   3. LLM recibe el resultado")
        print("   4. LLM genera respuesta con el resultado\n")
        
        result = await agent.run(task)
        
        print("\n" + "="*70)
        print("RESULTADO")
        print("="*70)
        print(f"\n🤖 Respuesta: {result.text}")
        print("\n✅ Tarea completada exitosamente")

print("✅ Función main() definida")
print("\n⚠️ Ejecuta la siguiente celda para correr el agente")

In [ ]:
# Ejecutar el agente
await main()

## 5. Pruebas con Diferentes Tareas

Vamos a probar el agente con múltiples tareas de código:

In [ ]:
async def test_multiple_tasks():
    """Prueba el agente con múltiples tareas de código"""
    
    tasks = [
        "Calculate the sum of squares of numbers from 1 to 100.",
        "Generate the first 10 Fibonacci numbers.",
        "Find all prime numbers less than 50.",
        "Calculate the mean and standard deviation of [5, 10, 15, 20, 25, 30]."
    ]
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential, 
            project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
            model_deployment_name=os.getenv("MODEL")
        ).create_agent(
            name="CodingAgent",
            instructions="You are a helpful assistant that can write and execute Python code.",
            tools=HostedCodeInterpreterTool()
        ) as agent,
    ):
        print("\n" + "="*70)
        print("PRUEBAS CON MÚLTIPLES TAREAS")
        print("="*70)
        
        for i, task in enumerate(tasks, 1):
            print(f"\n{'─'*70}")
            print(f"Tarea {i}: {task}")
            print(f"{'─'*70}")
            
            result = await agent.run(task)
            print(f"\n🤖 Respuesta: {result.text}")

# await test_multiple_tasks()

## Conclusión

Este ejemplo demuestra:

### 1. **Code Interpreter Tool**
- `HostedCodeInterpreterTool`: Ejecutor de código seguro
- Sandbox aislado para ejecución
- Soporte para bibliotecas Python comunes
- Integración seamless con agentes

### 2. **Dynamic Code Generation**
- LLM genera código basado en la tarea
- Código adaptado al problema específico
- No necesitas escribir código manualmente

### 3. **Azure AI Agent Client**
- `AzureAIAgentClient` para crear agentes
- Integración con Azure AI Foundry
- Gestión de recursos con async context managers

### 4. **Tool Integration**
- Tools como parámetro en `create_agent()`
- Framework maneja la orquestación
- Resultados automáticamente integrados

### Aplicaciones Prácticas:

✅ **Cálculos matemáticos**: Factoriales, estadísticas, álgebra

✅ **Análisis de datos**: Procesamiento, agregación, estadísticas

✅ **Algoritmos**: Sorting, búsqueda, optimización

✅ **Visualización**: Generar gráficos y plots

✅ **File processing**: CSV, JSON, XML parsing

✅ **Scientific computing**: NumPy, SciPy operations

### Bibliotecas disponibles típicamente:

- `math`: Funciones matemáticas
- `statistics`: Estadísticas básicas
- `numpy`: Computación numérica
- `pandas`: Análisis de datos
- `matplotlib`: Visualización
- `datetime`: Manejo de fechas
- `json`: Procesamiento JSON
- `re`: Regular expressions

### Comparación con otros enfoques:

| Enfoque | Flexibilidad | Seguridad | Complejidad |
|---------|-------------|-----------|-------------|
| Hardcoded functions | ❌ Baja | ✅ Alta | 😊 Baja |
| Code Interpreter | ✅✅ Muy Alta | ✅ Alta | 🤔 Media |
| Eval/Exec local | ✅✅ Muy Alta | ❌ Muy Baja | 😰 Alta |

### Ventajas de Code Interpreter:

✅ **Flexible**: Resuelve problemas no previstos

✅ **Seguro**: Sandbox aislado

✅ **Preciso**: Cálculos exactos vs aproximaciones del LLM

✅ **Escalable**: No necesitas predefinir todas las funciones

### Consideraciones de seguridad:

⚠️ **Sandbox**: Código ejecuta en entorno aislado

⚠️ **Timeouts**: Límites de tiempo de ejecución

⚠️ **Recursos**: Límites de memoria y CPU

⚠️ **Network**: Típicamente sin acceso a red

✅ **Audit**: Loggear código ejecutado para auditoría

### Limitaciones:

❌ **No file system persistente**: Archivos se pierden

❌ **No external APIs**: Sin acceso a servicios externos

❌ **Bibliotecas limitadas**: Solo las preinstaladas

❌ **Timeouts**: Operaciones muy largas pueden fallar

### Best Practices:

🔹 **Validar inputs**: Antes de ejecutar código

🔹 **Error handling**: Manejar excepciones de ejecución

🔹 **Logging**: Registrar código ejecutado

🔹 **Monitoring**: Track uso y performance

🔹 **Testing**: Probar con casos edge

### Extensiones posibles:

🔹 Agregar más herramientas especializadas

🔹 Implementar caching de resultados

🔹 Guardar código generado para reutilización

🔹 Visualización de resultados

🔹 Export de código a notebooks

🔹 Integration con Data Lakes